In [ ]:
import sqlite3
from datetime import datetime, timedelta

In [ ]:
#connect to local sqlite database (create if it is not existing)
conn = sqlite3.connect('office_data.db')
cursor = conn.cursor()

In [ ]:
#Create an attendance table
cursor.execute("""
create table if not exists attendance(
  id integer primary key autoincrement,
  name text,
  date text
)
""")

In [ ]:
#Insert some sample data

today = datetime.now()

for i in range(3):
  day = today - timedelta(days=i)
  for j in range(100 + i * 10):
    cursor.execute("insert into attendance(name,date) values (?,?)", (f"person_{j+1}" , day.strftime("%Y-%m-%d")))

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('office_data.db')
cursor = conn.cursor()

cursor.execute("select * from attendance limit 5")
rows = cursor.fetchall()
for row in rows:
  print(row)

conn.commit()
conn.close()

(1, 'person_1', '2025-07-22')
(2, 'person_2', '2025-07-22')
(3, 'person_3', '2025-07-22')
(4, 'person_4', '2025-07-22')
(5, 'person_5', '2025-07-22')


In [ ]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key = userdata.get('GEMINI_API_KEY'))

In [ ]:
gemini_model = genai.GenerativeModel('models/gemini-2.5-flash')

In [ ]:
def chatbot_query_handler(user_query):
  conn = sqlite3.connect('office_data.db')
  cursor = conn.cursor()

  #prompt for gemini to generate SQL query
  prompt = f"""
  Based on the following user query, Generate a SQLite SQL query to reterive information from the 'attendance' table.
  The 'attendance' table has columns 'id' (integer), 'name' (text), 'date' (text).
  The date format in the table is 'YYYY-MM-DD'.
  For example, If the user asks 'How many people were in the office yesterday?', the SQL query should be:
  SELECT count(*) from attendance where date = 'YYYY-MM-DD_for_yesterday'
  Replace 'YYYY-MM-DD_for_yesterday' with the actual date for yesterday.
  If the user asks for employees present on one day but not another, use a query with EXCEPT.
  If the user asks for attendance of a specific person, filter by name.
  You need to generate appropriate prompt based on user query that can be provide the result user is expecting from the 'attendance' table.

  User query : {user_query}
  """

  try:
    response = gemini_model.generate_content(prompt)
    generate_sql = response.text.replace("sqlite","").strip()
    generate_sql = generate_sql.replace("```","").strip()
    print(f"Generated SQL: {generate_sql}")

    conn.close()
    return generate_sql
  except Exception as e:
    conn.close()
    print(f"Bot: An error occured while generating SQL query with GEMINI: {e}")


In [ ]:
#Test the querries-
queries = [
    "How many people were in the office today?",
    "How many people were in the office on 2025-07-19?",
    "How many people were in the office yesterday?",
    "Show me the attendance for '2025-07-18",
    "Show me the attendance for '2025-07-20",
    "Tell me about those employees who present yesterday but not today",
    "Attendance report of person_101",
    "Tell me something else"
]

for query in queries:
  print(f"User: {query}")
  response = chatbot_query_handler(query)
  print(f"{response}")
  print()

User: How many people were in the office today?
Generated SQL: SELECT count(*) FROM attendance WHERE date = DATE('now')
SELECT count(*) FROM attendance WHERE date = DATE('now')

User: How many people were in the office on 2025-07-19?
Generated SQL: SELECT count(*) FROM attendance WHERE date = '2025-07-19'
SELECT count(*) FROM attendance WHERE date = '2025-07-19'

User: How many people were in the office yesterday?
Generated SQL: SELECT count(*) FROM attendance WHERE date = DATE('now', '-1 day');
SELECT count(*) FROM attendance WHERE date = DATE('now', '-1 day');

User: Show me the attendance for '2025-07-18
Generated SQL: SELECT * FROM attendance WHERE date = '2025-07-18'
SELECT * FROM attendance WHERE date = '2025-07-18'

User: Show me the attendance for '2025-07-20
Generated SQL: SELECT name FROM attendance WHERE date = '2025-07-20'
SELECT name FROM attendance WHERE date = '2025-07-20'

User: Tell me about those employees who present yesterday but not today
Generated SQL: SELECT name

In [ ]:
def chatbot_query_handler(user_query):
  conn = sqlite3.connect('office_data.db')
  cursor = conn.cursor()

  #prompt for gemini to generate SQL query
  prompt = f"""
  Based on the following user query, Generate a SQLite SQL query to reterive information from the 'attendance' table.
  The 'attendance' table has columns 'id' (integer), 'name' (text), 'date' (text).
  The date format in the table is 'YYYY-MM-DD'.
  For example, If the user asks 'How many people were in the office yesterday?', the SQL query should be:
  SELECT count(*) from attendance where date = 'YYYY-MM-DD_for_yesterday'
  Replace 'YYYY-MM-DD_for_yesterday' with the actual date for yesterday.
  If the user asks for employees present on one day but not another, use a query with EXCEPT.
  If the user asks for attendance of a specific person, filter by name.
  You need to generate appropriate prompt based on user query that can be provide the result user is expecting from the 'attendance' table.

  User query : {user_query}
  """

  try:
    response = gemini_model.generate_content(prompt)
    generate_sql = response.text.replace("sqlite","").strip()
    generate_sql = generate_sql.replace("```","").strip()
    print(generate_sql)

    cursor.execute(generate_sql)
    result = cursor.fetchall()

    conn.close()
    return result, len(result)
  except Exception as e:
    conn.close()
    print(f"Bot: An error occured while generating SQL query with GEMINI: {e}")


In [ ]:
#Test the querries-
queries = [
    "How many people were in the office today?",
    "How many people were in the office on 2025-07-19?",
    "How many people were in the office yesterday?",
    "Show me the attendance for '2025-07-18",
    "Show me the attendance for '2025-07-20",
    "Tell me about those employees who present yesterday but not today",
    "Attendance report of person_101",
    "Tell me something else"
]

for query in queries:
  print(f"User: {query}")
  response = chatbot_query_handler(query)
  print(f"{response}")
  print()

User: How many people were in the office today?
SELECT count(*) FROM attendance WHERE date = CURRENT_DATE;
([(100,)], 1)

User: How many people were in the office on 2025-07-19?
SELECT count(*) FROM attendance WHERE date = '2025-07-19'
([(0,)], 1)

User: How many people were in the office yesterday?
SELECT count(*) FROM attendance WHERE date = date('now', '-1 day');
([(110,)], 1)

User: Show me the attendance for '2025-07-18
SELECT id, name, date FROM attendance WHERE date = '2025-07-18';
([], 0)

User: Show me the attendance for '2025-07-20
SELECT name FROM attendance WHERE date = '2025-07-20'
([('person_1',), ('person_2',), ('person_3',), ('person_4',), ('person_5',), ('person_6',), ('person_7',), ('person_8',), ('person_9',), ('person_10',), ('person_11',), ('person_12',), ('person_13',), ('person_14',), ('person_15',), ('person_16',), ('person_17',), ('person_18',), ('person_19',), ('person_20',), ('person_21',), ('person_22',), ('person_23',), ('person_24',), ('person_25',), ('per

In [ ]:
def chatbot_query_handler(user_query):
  conn = sqlite3.connect('office_data.db')
  cursor = conn.cursor()

  #prompt for gemini to generate SQL query
  prompt = f"""
  Based on the following user query, Generate a SQLite SQL query to reterive information from the 'attendance' table.
  The 'attendance' table has columns 'id' (integer), 'name' (text), 'date' (text).
  The date format in the table is 'YYYY-MM-DD'.
  For example, If the user asks 'How many people were in the office yesterday?', the SQL query should be:
  SELECT count(*) from attendance where date = 'YYYY-MM-DD_for_yesterday'
  Replace 'YYYY-MM-DD_for_yesterday' with the actual date for yesterday.
  If the user asks for employees present on one day but not another, use a query with EXCEPT.
  If the user asks for attendance of a specific person, filter by name.
  You need to generate appropriate prompt based on user query that can be provide the result user is expecting from the 'attendance' table.

  User query : {user_query}
  """

  try:
    response = gemini_model.generate_content(prompt)
    generate_sql = response.text.replace("sqlite","").strip()
    generate_sql = generate_sql.replace("```","").strip()
    print(generate_sql)

    cursor.execute(generate_sql)
    result = cursor.fetchall()

    if len(result) == 0:
      return "No Information"
    elif len(result) > 1:
      return result
    else:
      return result[0]

    conn.close()
  except Exception as e:
    conn.close()
    print(f"Bot: An error occured while generating SQL query with GEMINI: {e}")


In [ ]:
#Test the querries-
queries = [
    "How many people were in the office today?",
    "How many people were in the office on 2025-07-19?",
    "How many people were in the office yesterday?",
    "Show me the attendance for '2025-07-18",
    "Show me the attendance for '2025-07-20",
    "Tell me about those employees who present yesterday but not today",
    "Attendance report of person_101",
    "Tell me something else"
]

for query in queries:
  print(f"User: {query}")
  response = chatbot_query_handler(query)
  print(f"{response}")
  print()

User: How many people were in the office today?
SELECT count(*) FROM attendance WHERE date = date('now');
(100,)

User: How many people were in the office on 2025-07-19?
SELECT count(*) FROM attendance WHERE date = '2025-07-19';
(0,)

User: How many people were in the office yesterday?
SELECT count(*) FROM attendance WHERE date = date('now', '-1 day');
(110,)

User: Show me the attendance for '2025-07-18
SELECT * FROM attendance WHERE date = '2025-07-18';
No Information

User: Show me the attendance for '2025-07-20
SELECT name FROM attendance WHERE date = '2025-07-20';
[('person_1',), ('person_2',), ('person_3',), ('person_4',), ('person_5',), ('person_6',), ('person_7',), ('person_8',), ('person_9',), ('person_10',), ('person_11',), ('person_12',), ('person_13',), ('person_14',), ('person_15',), ('person_16',), ('person_17',), ('person_18',), ('person_19',), ('person_20',), ('person_21',), ('person_22',), ('person_23',), ('person_24',), ('person_25',), ('person_26',), ('person_27',), 